In [44]:
import os
import tensorflow as tf
import numpy as np


# Check that we have correct TensorFlow version installed
tf_version = tf.__version__
print("TensorFlow version: {}".format(tf_version))
assert "1.5" <= tf_version, "TensorFlow r1.5 or later is needed"

TensorFlow version: 1.5.0


In [45]:
tf.logging.set_verbosity(tf.logging.INFO)

train_file = "regression-train.csv"
test_file = "regression-test.csv"

In [46]:
numerical_feature_names = [
    'PctUnder18',
    'PctOver65',
    'PctFemale',
    'PctWhite',
    'PctBachelors',
    'PctDem',
    'PctGop'
]

feature_columns = [tf.feature_column.numeric_column(k) for k in numerical_feature_names]

def my_input_fn(file_path, repeat_count=200):
    def decode_csv(line):
        parsed_line = tf.decode_csv(line, [[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.]])
        label = parsed_line[-1]  # Last element is the label
        features = parsed_line[:-1] # Everything but last elements are the features
        d = dict(zip(numerical_feature_names, features)), label
        return d

    dataset = (tf.data.TextLineDataset(file_path)  # Read text file
               .map(decode_csv))  # Transform each elem by applying decode_csv fn
    dataset = dataset.shuffle(buffer_size=256)
    dataset = dataset.repeat(repeat_count)  # Repeats dataset this # times
    dataset = dataset.batch(8)  # Batch size to use
    return dataset

In [53]:
classifier = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# Run training for 7 epochs (7 times through our entire dataset)
# You can experiment with this value for your own dataset
classifier.train(
    input_fn=lambda: my_input_fn(train_file, 7))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/1h/g9jk9_kx67d6g0_gyfnvk1n4008m_k/T/tmpinbxdb8_', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x121dc8e80>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/1h/g9jk9_kx67d6g0_gyfnvk1n4008m_k/T/tmpinbxdb8_/model.ckpt.
INFO:tensorflow:loss = 1.0388168, step = 1
INFO:tensorflow:global_step/sec: 630.799
INFO:tensorflow:loss = 0.19403698, step = 101 (0.159 sec)
INFO:tensorflow:global_step/sec: 815.889
INFO:tensorflow:loss = 0.074764445

In [54]:
results = classifier.evaluate(input_fn=lambda: my_input_fn(test_file, 1))
for key in sorted(results):
  print('%s: %s' % (key, results[key]))

INFO:tensorflow:Starting evaluation at 2018-02-22-18:04:47
INFO:tensorflow:Restoring parameters from /var/folders/1h/g9jk9_kx67d6g0_gyfnvk1n4008m_k/T/tmpinbxdb8_/model.ckpt-2188
INFO:tensorflow:Finished evaluation at 2018-02-22-18:04:47
INFO:tensorflow:Saving dict for global step 2188: average_loss = 0.001390136, global_step = 2188, loss = 0.011048873
average_loss: 0.001390136
global_step: 2188
loss: 0.011048873


In [55]:
# Generate predictions on 3 counties
prediction_input = {
    'PctUnder18': [23.9, 25.7, 10.6],
    'PctOver65': [17.6,24.7,15.8],
    'PctFemale': [50.0,48.5,53.5],
    'PctWhite':[0.965, 0.97, 0.75],
    'PctBachelors':[12.7, 17.0, 49.8],
    'PctDem': [0.3227832512315271, 0.09475032010243278, 0.6346801346801347],
    'PctGop': [0.6545566502463054, 0.8911651728553138, 0.3468013468013468]
}

def test_input_fn():
   dataset = tf.data.Dataset.from_tensors(prediction_input)
   return dataset

# Predict all our prediction_input
pred_results = classifier.predict(input_fn=test_input_fn)

In [56]:
# Actual values for the raw prediction data:
# 1) 23% Clinton
# 2) 5% Clinton
# 3) 69% Clinton

for pred in enumerate(pred_results):
    print(pred)

INFO:tensorflow:Restoring parameters from /var/folders/1h/g9jk9_kx67d6g0_gyfnvk1n4008m_k/T/tmpinbxdb8_/model.ckpt-2188
(0, {'predictions': array([0.2305259], dtype=float32)})
(1, {'predictions': array([0.03020383], dtype=float32)})
(2, {'predictions': array([0.68529695], dtype=float32)})
